# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('../output_data/cities.csv')
df.head()

,Unnamed: 0,City,Country,Longitude,Latitude,Humidity (%),Wind Speed (mph),Cloudiness (%),Max Temp (f)
0,0,Nikolskoye,RU,30.79,59.70,97,3.29,100,30.09
1,1,Lakes Entrance,AU,147.98,-37.88,90,5.26,6,57.58
2,2,Jiuquan,CN,98.52,39.74,70,4.52,29,20.01
3,3,Iqaluit,CA,-68.51,63.75,70,13.80,100,-2.31
4,4,Chernyshevskiy,RU,112.47,63.01,97,4.61,100,-17.37


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
coordinates = []
for idx, city in df.iterrows():
    pair = []
    pair.append(round(city['Latitude'], 2))
    pair.append(round(city['Longitude'], 2))
    coordinates.append(pair)
 
weight = df['Humidity (%)'].astype(float)
coordinates

[[59.7, 30.79],
 [-37.88, 147.98],
 [39.74, 98.52],
 [63.75, -68.51],
 [63.01, 112.47],
 [19.73, -155.09],
 [42.6, -73.97],
 [47.92, 9.77],
 [55.74, -97.86],
 [14.9, -24.5],
 [3.07, 172.79],
 [-9.8, -139.03],
 [-23.12, -134.97],
 [-24.79, -50.01],
 [38.42, -121.42],
 [-42.78, 147.06],
 [-16.48, -151.75],
 [-1.68, 29.22],
 [-53.15, -70.92],
 [66.8, 65.81],
 [35.06, 25.23],
 [61.69, 27.27],
 [-54.8, -68.3],
 [-33.65, 115.33],
 [68.22, -135.01],
 [43.32, 145.57],
 [68.05, 39.51],
 [73.51, 80.55],
 [40.16, -8.33],
 [37.47, -4.44],
 [-34.53, 20.04],
 [37.75, -100.02],
 [58.0, 102.66],
 [-21.2, -175.2],
 [77.48, -69.36],
 [-34.42, 19.23],
 [46.07, 30.47],
 [30.15, 62.6],
 [-19.55, 169.27],
 [36.88, 118.74],
 [34.17, -82.12],
 [16.28, 95.68],
 [-33.7, -53.46],
 [-3.8, 102.27],
 [37.1, -113.58],
 [65.61, -37.64],
 [5.56, 95.32],
 [69.22, -51.1],
 [70.63, 147.92],
 [-2.02, 147.27],
 [66.15, -65.71],
 [5.89, 95.32],
 [-5.0, 34.73],
 [21.35, -158.09],
 [78.22, 15.64],
 [-33.59, 26.89],
 [14.31, 1

In [4]:
fig = gmaps.figure()
heat_map = gmaps.heatmap_layer(coordinates,
                            weights=weight,
                            dissipating=False, 
                            point_radius=5
                              )
fig.add_layer(heat_map)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_df = df.loc[(df['Wind Speed (mph)'] < 10) &
                     ((df['Max Temp (f)'] > 70) & (df['Max Temp (f)'] < 95)) &
                     (df['Cloudiness (%)'] < 30), :]
hotel_df = narrowed_df.dropna()
hotel_df

,Unnamed: 0,City,Country,Longitude,Latitude,Humidity (%),Wind Speed (mph),Cloudiness (%),Max Temp (f)
9,9,São Filipe,CV,-24.50,14.90,69,8.12,0,73.92
35,35,Hermanus,ZA,19.23,-34.42,75,9.46,3,76.89
41,41,Pyapon,MM,95.68,16.28,53,5.82,7,71.11
53,53,Makakilo City,US,-158.09,21.35,70,5.75,0,71.92
66,66,Ellisras,ZA,27.74,-23.66,23,7.99,8,93.07
68,68,Bindura,ZW,31.33,-17.30,80,3.27,27,73.89
96,96,Avarua,CK,-159.78,-21.21,83,8.05,20,77.05
109,109,Am Timan,TD,20.28,11.03,8,4.12,2,92.21
116,116,Puerto Ayora,EC,-90.35,-0.74,73,6.80,19,78.76
160,160,Phayakkhaphum Phisai,TH,103.19,15.52,36,6.76,11,85.78


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name'] = ""
hotel_df.head()

C:\Users\Billy\AppData\Local\Temp/ipykernel_19928/341898012.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,Unnamed: 0,City,Country,Longitude,Latitude,Humidity (%),Wind Speed (mph),Cloudiness (%),Max Temp (f),Hotel Name
9,9,São Filipe,CV,-24.50,14.90,69,8.12,0,73.92,
35,35,Hermanus,ZA,19.23,-34.42,75,9.46,3,76.89,
41,41,Pyapon,MM,95.68,16.28,53,5.82,7,71.11,
53,53,Makakilo City,US,-158.09,21.35,70,5.75,0,71.92,
66,66,Ellisras,ZA,27.74,-23.66,23,7.99,8,93.07,


In [7]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {'radius': '5000',
          'type': 'lodging',
          'key': g_key}
#response = requests.get(base_url, params=params).json()
#pprint(response['results'][0]['name'])
c = 0
for idx, row in hotel_df.iterrows():   
    try:
        params['location'] = f"{row['Latitude']},{row['Longitude']}"
        response = requests.get(base_url, params=params).json()
        hotel_df.loc[idx, 'Hotel Name'] = response['results'][0]['name']
        print('----------')
        print(f"{idx}: Storing Results for {row['City']}")
    except:
        print('----------')
        print("Missing field/result... skipping.")
        c = c + 1
hotel_df

C:\Users\Billy\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


----------
9: Storing Results for São Filipe
----------
35: Storing Results for Hermanus
----------
41: Storing Results for Pyapon
----------
53: Storing Results for Makakilo City
----------
66: Storing Results for Ellisras
----------
68: Storing Results for Bindura
----------
96: Storing Results for Avarua
----------
109: Storing Results for Am Timan
----------
116: Storing Results for Puerto Ayora
----------
160: Storing Results for Phayakkhaphum Phisai
----------
165: Storing Results for Kadapa
----------
172: Storing Results for Hualmay
----------
179: Storing Results for Grand Gaube
----------
210: Storing Results for Nakhon Phanom
----------
216: Storing Results for Belém de São Francisco
----------
219: Storing Results for Pisco
----------
220: Storing Results for Nicoya
----------
223: Storing Results for Arlit
----------
231: Storing Results for Bilma
----------
243: Storing Results for Necochea
----------
263: Storing Results for Olinda
----------
281: Storing Results for Geo

,Unnamed: 0,City,Country,Longitude,Latitude,Humidity (%),Wind Speed (mph),Cloudiness (%),Max Temp (f),Hotel Name
9,9,São Filipe,CV,-24.50,14.90,69,8.12,0,73.92,Hotel Xaguate
35,35,Hermanus,ZA,19.23,-34.42,75,9.46,3,76.89,Misty Waves Boutique Hotel
41,41,Pyapon,MM,95.68,16.28,53,5.82,7,71.11,Aung Naing Thu Motel
53,53,Makakilo City,US,-158.09,21.35,70,5.75,0,71.92,Marriott's Ko Olina Beach Club
66,66,Ellisras,ZA,27.74,-23.66,23,7.99,8,93.07,Lapa le le guesthouse
68,68,Bindura,ZW,31.33,-17.30,80,3.27,27,73.89,Mayfair Lodges and Restaurant
96,96,Avarua,CK,-159.78,-21.21,83,8.05,20,77.05,Paradise Inn
109,109,Am Timan,TD,20.28,11.03,8,4.12,2,92.21,Palais du chef Canton Salamt
116,116,Puerto Ayora,EC,-90.35,-0.74,73,6.80,19,78.76,Finch Bay Galapagos Hotel
160,160,Phayakkhaphum Phisai,TH,103.19,15.52,36,6.76,11,85.78,โรงแรมบ้านนงนุชโฮเตล


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
vaca_loc = []
for idx, city in hotel_df.iterrows():
    pair = []
    pair.append(round(city['Latitude'], 2))
    pair.append(round(city['Longitude'], 2))
    vaca_loc.append(pair)
markers = gmaps.marker_layer(vaca_loc, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))